# Solución etapa 3 - Training

In [ ]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl

In [ ]:
version_to_load='v1'

In [ ]:
# Cargar el dataset
import pandas as pd
csv_path = f'../data/preprocessed/preprocessed_data_{version_to_load}.csv'
print('... Loading:', csv_path,'...')
final_data = pd.read_csv(csv_path)

tabl(final_data)

In [ ]:
target_column = 'Fallo'

## Preprocesar

In [ ]:
# Dividir en variables predictoras (X) y objetivo (y)
X = final_data.drop(columns=[target_column])

y = final_data[target_column]

print(titl('X:'))
tabl(X)

In [ ]:
# Dividir en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Candidatos

1. LogisticRegression
2. GradientBoostingClassifier
3. SVC (kernel='rbf')

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.compose import make_column_selector as selector


### Pipelines

In [ ]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)


print(headr("Numerical columns"), numerical_columns)
print(headr("Categorical columns"), categorical_columns)

In [ ]:
# Preprocesamiento para variables categóricas
categorical_transformer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocesamiento para variables continuas
numerical_transformer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Combinar preprocesadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

In [ ]:
pipelines = {
    "LogisticRegression": Pipeline([('preprocessor', preprocessor),('classifier', LogisticRegression())]),
    # "GradientBoostingClassifier": Pipeline([('preprocessor', preprocessor),('classifier', GradientBoostingClassifier())]),
    # "SVC": Pipeline([('preprocessor', preprocessor),('classifier', SVC(kernel='linear'))]),
    # "SGDClassifier": Pipeline([('preprocessor', preprocessor),('classifier', SGDClassifier())]),
}

In [ ]:
# Cargamos funciones para best_model
from utils.best_model_utils import generate_learning_curves, generate_validation_curves, cv_train

### CV

In [ ]:
# Config CV
cvss = ShuffleSplit(n_splits=40, test_size=0.2, random_state=0)

In [ ]:
# Aplicar CV
results = {}

for name, pipeline in pipelines.items():
    print(headr(f"Entrenar {name}"))
    results[name] = cv_train(name, pipeline, cvss, X_train, y_train, X_test, y_test)

print(titl("Resultados:"))
results_df=pd.DataFrame(results)
tabl(results_df)

### Curva de aprendizaje


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Aplicar LC
train_sizes = np.linspace(0.1, 1.0, num=5, endpoint=True)

for pipeline_name, pipeline_obj in pipelines.items():
    generate_learning_curves(pipeline_name, pipeline_obj, X, y, train_sizes, cvss)

### Curva validación

In [ ]:
pname = 'LogisticRegression'
Cs = [0.1, 1, 10, 100]
# Cs = [5, 10, 15]
generate_validation_curves(pname, pipelines[pname], X, y, 'classifier__C', Cs, cvss)

## Afinar hiperparámentros

#### LogisticRegression

In [ ]:
from sklearn.model_selection import GridSearchCV

pname = 'LogisticRegression'

param_grid = {
    'classifier__C': [5, 10, 15],
    # 'classifier__penalty': ['l1', 'l2', 'elasticnet', None],
    'classifier__solver': ['lbfgs', 'liblinear', 'newto-cg', 'newto-cholesky', 'sag', 'saga']
    # 'classifier__max_iter': [1,10,100,1000]
}

grid_search = GridSearchCV(
    pipelines[pname], param_grid, cv=cvss, scoring="accuracy")

grid_search.fit(X_train, y_train)

print(headr(pname))
print("Mejores hiperparámetros:", grid_search.best_params_)
print(f"Mejor accuracy: {grid_search.best_score_:.2f}")

### re-entrenar + re-evaluar


In [ ]:
# Definir pipelines con mejores hiperparámetros
pipelines = {
    "LogisticRegression": Pipeline([('preprocessor', preprocessor),('classifier', LogisticRegression(C=5,solver='lbfgs'))]),
    # "GradientBoostingClassifier": Pipeline([('preprocessor', preprocessor),('classifier', GradientBoostingClassifier())]),
    "SVC": Pipeline([('preprocessor', preprocessor),('classifier', SVC(kernel='linear'))]),
    "SGDClassifier": Pipeline([('preprocessor', preprocessor),('classifier', SGDClassifier())]),
}


# Aplicar CV
results = {}

for name, pipeline in pipelines.items():
    print(headr(f"Entrenar {name}"))
    results[name] = cv_train(name, pipeline, cvss, X_train, y_train, X_test, y_test)

## Comparar resultados

In [ ]:
print(titl("Resultados:"))
results_df=pd.DataFrame(results)
tabl(results_df)

In [ ]:
# Guardar resultado
result_path= f'../results/result_{version_to_load}.csv'
results_df.to_csv(result_path, index=False)

## Desición
> Ninguno

- Muy por debajo de lo que se necesita
- Una columna con un peso descompensado en el resultado
- Se debe mejorar enfocando hacia interrelaciones a nivel de mes